In [2]:
CommentFolder = 'flat20To100mln'

In [3]:
#Необходимые бибилиотеки
import warnings
warnings.simplefilter("ignore")

from bs4 import BeautifulSoup #для загрузки страницы
from requests.exceptions import HTTPError #для идентификации прогрузки капчи
import cloudscraper #для загрузки страницы
import time #для остановок при запуске парсера
import re #для поиска HTML по тексту
import httpx #для поиска HTML по тексту

#Библиотеки для работы с данными
import pandas as pd #для выгрузки данных в .xlsx

In [4]:
#Блок загрузки HTML кодов страниц
#https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=' 

UrlWithoutPage ='https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=100000000&minarea=200&minfloor=1&minprice=20000000&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=5&region=1&p='

UrlWithoutPage ='https://www.cian.ru/cat.php?engine_version=2&offer_type=flat&deal_type=sale&region=1&p=1'

PageCounts = 54


k = 0
str_soups = []
Info2strAll = []
for i in range(1, PageCounts+1):
    WhereInfoStart = -1
    k = 0
    url = UrlWithoutPage + str(i)
    print('Загрузка страницы: ' + url)
    while WhereInfoStart == -1:
        #Спим чтобы незабанили
        time.sleep(4)       
        #Выгружаем HTML код страницы
        k = k + 1
        print('Попытка номер ' + str(k))        
        html = cloudscraper.create_scraper().get(url).text
        soup = BeautifulSoup(html)
        str_soup = str(soup)
        #Проверяем успешна ли прогрузка страницы (ищем специфичную строку)
        quote = "\'" #для одинарной кавычки в строке
        WhereInfoStart = str_soup.find('window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']')      
        if WhereInfoStart == -1:
            print('Неудачная попытка')
        else:
            print('Успешно')
            Info2strAll.append(str_soup)
        #На 10ой попытке выпрыгиваем из цикла    
        if k == 10:
            break  

Загрузка страницы: https://www.cian.ru/cat.php?engine_version=2&offer_type=flat&deal_type=sale&region=1&p=11
Попытка номер 1
Неудачная попытка
Попытка номер 2
Неудачная попытка
Попытка номер 3
Неудачная попытка
Попытка номер 4
Неудачная попытка
Попытка номер 5
Неудачная попытка
Попытка номер 6
Неудачная попытка
Попытка номер 7


KeyboardInterrupt: 

In [17]:
# Сохраняем строки в txt файлы
# Указываем путь папки, в которой создадим папку
PATH = #ваш путь ()
current_time_str = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(time.time()))

import os

name_folder = PATH + "CIAN_Pages_"+current_time_str+CommentFolder
current_time_str = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(time.time()))
# Создаем папку с уникальным названием (не раб)
if not os.path.exists(name_folder):
    # Создаем папку
    os.makedirs(name_folder)
    print("Папка успешно создана.")
else:
    print("Папка уже существует.")
#Выгружаем данные в текстовые файлы 
for i in range(0, len(Info2strAll)):
    name_text = name_folder + '\\Page_Number_'+str(i+1) + '_' + current_time_str+'.txt'
    with open(name_text, 'w', encoding='utf-8') as file:
        file.write(Info2strAll[i])
        
# Дополнительно выгружаем ссылку откуда спарсили данные      
name_text = name_folder + '\\info'+'.txt' 
with open(name_text, 'w', encoding='utf-8') as file:
        # Записываем строку в файл
        file.write(UrlWithoutPage)

Папка успешно создана.


In [5]:
#Вывод полного списка возможных переменные
NumberOfObservation = 9

infoVariables = []
page_HTML = Info2strAll[0]
dfpage = pd.DataFrame()
LengthOfDataOnPage = [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 56]

for k in range(1, 100):
    res = re.findall(r'"\w{'+str(k)+'}":', page_HTML)
    res = list(dict.fromkeys(res))
    for i in res:
        res = re.findall(i+r'[^,]+', page_HTML)
        pattern = re.compile(r'[^,]+')
        if len(res) in LengthOfDataOnPage:
            values = [pattern.search(s).group().replace(i, '') if pattern.search(s) else None for s in res]                        
            if len(values) < max(LengthOfDataOnPage)+1:
                values += [None] * abs(len(dfpage) - len(values))
            dfpage[i] = values
            infoVariables.append(i)

# Получить имена параметров
param_names = dfpage.columns.tolist()
# Получить значения параметров в первой строке
param_values = dfpage.iloc[NumberOfObservation].tolist()
# Получить длины значений параметров
param_lengths = [len(str(value)) for value in param_values]
# Получить количество значений параметров
param_counts = [len(dfpage[column].dropna()) for column in param_names]
# Вывести имена, значения, длины значений и количество значений параметров
# Создание DataFrame из списка имен параметров, значений, длин значений и количества значений
data = {'Имя параметра': param_names,
        'Значение параметра в ' + str(NumberOfObservation) + ' наблюдении': param_values,
        'Длина значения': param_lengths,
        'Количество значений': param_counts}
df_tmp = pd.DataFrame(data)
df_tmp = df_tmp.sort_values(by='Имя параметра')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_tmp)

,Имя параметра,Значение параметра в 9 наблюдении,Длина значения,Количество значений
94,"""accessType"":","""passSystem""",12,56
11,"""added"":","""2 июл",6,28
154,"""addedTimestamp"":",1688291774,10,28
175,"""additionalTypes"":",[],2,28
38,"""address"":","[{""title"":""Москва""",18,32
125,"""adfoxParams"":","{""puid1"":""deal_type_sale""",25,28
105,"""agencyName"":","""ID: 73831686""",14,27
181,"""agentAccountType"":","""specialist""}",13,28
192,"""agentAvailability"":","{""available"":true",17,28
99,"""agentBonus"":",null,4,28


In [18]:
#Вывод полного списка возможных переменные
NumberOfObservation = 10

infoVariables = []
page_HTML = Info2strAll[0]
dfpage = pd.DataFrame()
LengthOfDataOnPage = [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 56]

for k in range(1, 100):
    res = re.findall(r'"\w{'+str(k)+'}":', page_HTML)
    res = list(dict.fromkeys(res))
    for i in res:
        res = re.findall(i+r'[^,]+', page_HTML)
        pattern = re.compile(r'[^,]+')
        if len(res) in LengthOfDataOnPage:
            values = [pattern.search(s).group().replace(i, '') if pattern.search(s) else None for s in res]                        
            if len(values) < max(LengthOfDataOnPage)+1:
                values += [None] * abs(len(dfpage) - len(values))
            dfpage[i] = values
            infoVariables.append(i)

# Получить имена параметров
param_names = dfpage.columns.tolist()
# Получить значения параметров в первой строке
param_values = dfpage.iloc[NumberOfObservation].tolist()
# Получить длины значений параметров
param_lengths = [len(str(value)) for value in param_values]
# Получить количество значений параметров
param_counts = [len(dfpage[column].dropna()) for column in param_names]
# Вывести имена, значения, длины значений и количество значений параметров
# Создание DataFrame из списка имен параметров, значений, длин значений и количества значений
data = {'Имя параметра': param_names,
        'Значение параметра в ' + str(NumberOfObservation) + ' наблюдении': param_values,
        'Длина значения': param_lengths,
        'Количество значений': param_counts}
df_tmp = pd.DataFrame(data)
df_tmp = df_tmp.sort_values(by='Имя параметра')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_tmp)


,Имя параметра,Значение параметра в 10 наблюдении,Длина значения,Количество значений
99,"""accessType"":",null,4,56
127,"""accountType"":","""managementCompany""",19,56
12,"""added"":","""сегодня",8,28
163,"""addedTimestamp"":",1689149324,10,28
179,"""additionalTypes"":",[]},3,28
40,"""address"":","[{""locationTypeId"":1",20,32
129,"""adfoxParams"":","{""puid1"":""deal_type_sale""",25,28
111,"""agencyName"":","""А101 БК Прокшино""",18,26
184,"""agentAccountType"":","""agency""",8,28
196,"""agentAvailability"":","{""available"":true",17,28


## Скраббируем данные

In [20]:
#Найдем номер конечной страницы
tmp = 0
NumberOfEndPage = 0
while tmp == 0:
    NumberOfEndPage = NumberOfEndPage + 1
    tmp = len(re.findall(r'Дополнительные предложения по вашему запросу', Info2strAll[NumberOfEndPage]))
    
NumberOfEndPage

53

In [ ]:
#Cкраббируем данные
df = pd.DataFrame()

for i in range(1, 2 + NumberOfEndPage):       
    NumberOfAds = 28
    if i == NumberOfEndPage+1:
        NumberOfAds = len(re.findall(r'"priceRur":[\d]+,', Info2strAll[NumberOfEndPage]))
        
    Info2str = Info2strAll[i-1]
    print('sheep')
    dfpage = pd.DataFrame()
    cianId_res_extracted = []
    cianUserId_res_extracted = []
    priceRur_res_extracted = []
    totalArea_res_extracted = []
    floorNumber_res_extracted = []
    creationDate_res_extracted = []
    coordinates_res_extractedLat = []
    coordinates_res_extractedLng = []
    districts_res_extracted = []
    buildYearres_extracted  = []
    addedTimestamp_res_extracted = []
    
    res = re.findall(r'"addedTimestamp":[^"]+,', page_HTML)
    pattern = re.compile(r'"addedTimestamp":[^,]+')
    addedTimestamp_res_extracted = [pattern.search(s).group().replace('"addedTimestamp":', '') if pattern.search(s) else None for s in res]
    dfpage['Added Time'] = addedTimestamp_res_extracted[0:NumberOfAds]
    # Преобразование формата времени
    dfpage['Added Time'] = pd.to_datetime(dfpage['Added Time'], unit='s')

    res = re.findall(r'"cianId":["\d]+', Info2str)
    pattern = re.compile(r'[\d]+')
    CianUrl = 'www.cian.ru/sale/commercial/'
    cianId_res_extracted = [CianUrl+pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Id Promotion'] = cianId_res_extracted[0:NumberOfAds]

    res = re.findall(r'"cianUserId":[\d]+,', Info2str)
    pattern = re.compile(r'[\d]+')
    cianUserId_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Id User'] = cianUserId_res_extracted[0:NumberOfAds]

    res = re.findall(r'"priceRur":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    priceRur_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Full Price'] = priceRur_res_extracted[0:NumberOfAds]

    res = re.findall(r'"totalArea":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    totalArea_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Total Area'] = totalArea_res_extracted[1:1+NumberOfAds]

    res = re.findall(r'"floorNumber":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    floorNumber_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Floor Number'] = floorNumber_res_extracted[0:NumberOfAds]

    res = re.findall(r'"creationDate":[^,]+', Info2str)
    pattern = re.compile(r'[-\d]+')
    creationDate_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Creation Date'] = creationDate_res_extracted[0:NumberOfAds]

    res = re.findall(r'"coordinates":[.:"{\w\d]+,[.:"{\w\d]+', Info2str)
    patternLat = re.compile(r'"lat":[.\d]+')
    patternLng = re.compile(r'"lng":[.\d]+')
    coordinates_res_extractedLat = [patternLat.search(s).group().replace('"lat":', '') if pattern.search(s) else None for s in res]
    coordinates_res_extractedLng = [patternLng.search(s).group().replace('"lng":', '') if pattern.search(s) else None for s in res]
    dfpage['Lat'] = coordinates_res_extractedLat[0:NumberOfAds]
    dfpage['Lng'] = coordinates_res_extractedLng[0:NumberOfAds]

    res = re.findall(r'"districts":\[\{[^}]+', Info2str)
    pattern = re.compile(r'"fullName":[^,]+')
    districts_res_extracted = [pattern.search(s).group().replace('"fullName":', '') if pattern.search(s) else None for s in res]
    #dfpage['District'] = districts_res_extracted[0:NumberOfAds]

    res = re.findall(r'"buildYear":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    buildYearres_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Build Year'] = buildYearres_extracted[0:NumberOfAds]

    res = re.findall(r'"description":"[^"]+', Info2str)
    pattern = re.compile(r'"description":"[^"]+')
    description_extracted = [pattern.search(s).group().replace('"description":', '') if pattern.search(s) else None for s in res]
    dfpage['Description'] = description_extracted[3:3+NumberOfAds]  
        
    res = re.findall(r'"floorsCount":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    floorsCount_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Floor Count'] = floorsCount_res_extracted[0:NumberOfAds]
     
    res = re.findall(r'"readyBusinessType":[^,]+', Info2str)
    pattern = re.compile(r'"readyBusinessType":[^,]+')
    readyBusinessType_extracted = [pattern.search(s).group().replace('"readyBusinessType":', '') if pattern.search(s) else None for s in res]
    dfpage['Ready Business?'] = readyBusinessType_extracted[0:NumberOfAds]
    
    res = re.findall(r'"materialType":[^,]+', Info2str)
    pattern = re.compile(r'"materialType":[^,]+')
    materialType_extracted = [pattern.search(s).group().replace('"materialType":', '') if pattern.search(s) else None for s in res]
    dfpage['Material Type'] = materialType_extracted[0:NumberOfAds]

    data = [df, dfpage]
    df = pd.concat(data)
    dfpage = 0

    
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):     
    #display(df) 
    
df.to_excel(name_folder+'\\'+"Df"+current_time_str+".xlsx")

In [ ]:
res = re.findall(r'"addedTimestamp":[^"]+,', page_HTML)
pattern = re.compile(r'"addedTimestamp":[^,]+')
cianUserId_res_extracted = [pattern.search(s).group().replace('"addedTimestamp":', '') if pattern.search(s) else None for s in res]
dfpage['Added Time'] = addedTimestamp_res_extracted
